In [ ]:
!git clone https://github.com/seujung/KoBART-summarization

In [ ]:
!pip install transformers

In [ ]:
!pip install -r /home/alpaco/hw/KoBART-summarization/requirements.txt

In [ ]:
!pip install loguru

# 데이터

In [ ]:
%cd /home/alpaco/hw/KoBART-summarization/data

/home/alpaco/hw/KoBART-summarization/data


In [2]:
!gdown 1b_XKu4-N_jlTI4YfcVs-cAcAwRYZLBnc

Downloading...
From: https://drive.google.com/uc?id=1b_XKu4-N_jlTI4YfcVs-cAcAwRYZLBnc
To: /home/alpaco/hw/KoBART-summarization/data/train.csv
100%|██████████████████████████████████████| 15.5M/15.5M [00:00<00:00, 25.0MB/s]


# 원 데이터 형식 확인

In [ ]:
!tar -zxvf /content/KoBART-summarization/data/test.tar.gz -C /content/KoBART-summarization/data

test.tsv


In [ ]:
!tar -zxvf /content/KoBART-summarization/data/train.tar.gz -C /content/KoBART-summarization/data

train.tsv


In [ ]:
import torch

# GPU 0만 사용하도록 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 모델과 데이터를 GPU로 이동
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

# 수제 데이터 csv to tsv

In [2]:
import pandas as pd

#train
# delimiter가 콤마(,)로 된 csv파일을 불러온 후(한글이라 인코딩은 utf-8 사용)
df = pd.read_csv('/home/alpaco/hw/KoBART-summarization/data/train.csv', sep = ",", engine='python', encoding = "cp949") #cp949

# 결측치 제거
df.dropna(axis=0, inplace=True)

# delimeter를 탭(\t)으로 변경 후 저장.
df.to_csv('/home/alpaco/hw/KoBART-summarization/data/train.tsv', sep='\t',encoding = "utf-8", index=False)

In [3]:
#test
# delimiter가 콤마(,)로 된 csv파일을 불러온 후(한글이라 인코딩은 utf-8 사용)
df = pd.read_csv('/home/alpaco/hw/KoBART-summarization/data/test.csv', sep = ",", engine='python', encoding = "cp949") #utf-8-sig

# 결측치 제거
df.dropna(axis=0, inplace=True)

# delimeter를 탭(\t)으로 변경 후 저장.
df.to_csv('/home/alpaco/hw/KoBART-summarization/data/test.tsv', sep='\t',encoding = "utf-8", index=False)

# train

In [1]:
#입력창 안 뜨는 문제로 wandb 사용이 어려울 시
!export WANDB_MODE="disabled"

In [4]:
!CUDA_VISIBLE_DEVICES=0 python /home/alpaco/hw/KoBART-summarization/train.py --gradient_clip_val 1.0 \
                --train_file '/home/alpaco/hw/KoBART-summarization/data/train.tsv' \
                --test_file '/home/alpaco/hw/KoBART-summarization/data/test.tsv' \
                --max_epochs 1001 \
                --checkpoint checkpoint \
                --accelerator gpu \
                --num_gpus 1 \
                --batch_size 26 \
                --num_workers 4
                
#CUDA_VISIBLE_DEVICES=0: 0번 gpu 사용 시

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
2023-10-17 14:12:52.590 | INFO     | __main__:<module>:75 - Namespace(accelerator='gpu', batch_size=26, checkpoint='checkpoint', gradient_clip_val=1.0, lr=3e-05, max_epochs=1001, max_len=512, num_gpus=1, num_workers=4, test_file='/home/alpaco/hw/KoBART-summarization/data/test.tsv', train_file='/home/alpaco/hw/KoBART-summarization/data/train.tsv')
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
wandb: Currently logged in as: yhw7012. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wand

In [ ]:
#러닝레이트 조정

!CUDA_VISIBLE_DEVICES=0 python /home/alpaco/hw/KoBART-summarization/train.py --gradient_clip_val 1.0 \
                --train_file '/home/alpaco/hw/KoBART-summarization/data/train.tsv' \
                --test_file '/home/alpaco/hw/KoBART-summarization/data/test.tsv' \
                --max_epochs 201 \
                --checkpoint checkpoint \
                --accelerator gpu \
                --num_gpus 1 \
                --lr 0.00005 \
                --batch_size 26 \
                --num_workers 64
                
# num_gpus: gpu 개수 4 -> 1로 변경
#배치사이즈 28 터짐, 26 돌아감
#1에폭에 5분 15초

In [ ]:
#재학습 코드

!CUDA_VISIBLE_DEVICES=0 python train.py --gradient_clip_val 1.0 \
                --max_epochs 1001 \
                # --default_root_dir logs \
                --num_gpus 1 \
                --batch_size 4 \
                --num_workers 4 \
                --resume_from_checkpoint /home/alpaco/hw/KoBART-summarization/checkpoint/model_chp/epoch=10-val_loss=2.144.ckpt
            

# 모델 저장

In [ ]:
# kobart_summary 디렉토리에 모델 저장
# hparams: logs 하위 디렉토리에서 사용할 모델의 버전 골라 hparams.yaml set
# model_binary: logs 하위 디렉토리에서 사용할 체크포인트 골라 *.ckpt set
!python get_model_binary.py --hparams /home/alpaco/hw/KoBART-summarization/lightning_logs/version_60/hparams.yaml --model_binary checkpoint/last-v6.ckpt

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# 인퍼런스

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained('/home/alpaco/hw/KoBART-summarization/kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

text = """
1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
"""

if text:
    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=4)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output)
    
    #인퍼런스 시간 30초
    #원래 모델: 1일 0 9시까지 최소 20만3220명이 코로나19에 신규 확진되어 역대 최다 기록을 갈아치웠다.
    #60, v3: 이날 0시부터 오후 6시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
    #59, v2: 2일 0시부터 오후 9시까지 집계된 확진자 수는 전날 같은 시간 기록한 13만3481명을 웃돌았다.
    #59, last-v1_1003_112epo: 이날 0시부터 오후 9시까지 전국 신규 확진자는 20만3220명으로 집계돼 동시간대 최다 기록도 갈아치웠다.

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
